In [15]:
from langchain_community.document_loaders import WebBaseLoader
import bs4
loader=WebBaseLoader(web_paths=("https://en.wikipedia.org/wiki/History_of_Egypt",),
    bs_kwargs=dict(parse_only=bs4.SoupStrainer(
        class_=('mw-body-content')
    )))
docs=loader.load()

# from unittest import loader
# from langchain_community.document_loaders import PyPDFLoader
# loader=PyPDFLoader("somatosensory.pdf")
# docs=loader.load()


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [16]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter=RecursiveCharacterTextSplitter(chunk_size=900 ,chunk_overlap=90)
final_split=splitter.split_documents(docs)
final_split

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/History_of_Egypt'}, page_content="This article's lead section may be too long. Please read the length guidelines and help move details into the article's body. (May 2023) (Learn how and when to remove this message)\n\n\nPart of a series on the\n\nHistory of Egypt\n\nPrehistoric Egypt Paleolithic300,000–17,000 BC Mesolithic17,000–9000 BC Predynastic Period6000–3000 BC\n\nAncient Egypt Early Dynastic Period3150–2686 BC Old Kingdom2686–2181 BC 1st Intermediate Period2181–2055 BC Middle Kingdom2055–1650 BC 2nd Intermediate Period1650–1550 BC New Kingdom1550–1069 BC 3rd Intermediate Period1069–664 BC Late Period664–332 BC\n\nGreco-Roman Egypt Argead dynasty332–310 BC Ptolemaic dynasties310–30 BC Roman and Byzantine Egypt30 BC–641 AD Sasanian Egypt619–629"),
 Document(metadata={'source': 'https://en.wikipedia.org/wiki/History_of_Egypt'}, page_content='Medieval Egypt Rashidun caliphate641–661 Umayyad caliphate661–750 Abbasid dynasty

In [17]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings=(
    OllamaEmbeddings(model="mxbai-embed-large:335m")
)

In [7]:

# text=[doc.page_content for doc in final_split]
# embeded=embeddings.embed_documents(text)
# embeded

In [18]:
from langchain_community.vectorstores import FAISS
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
faiss_store=FAISS.from_documents(final_split,embeddings)
# faiss_store.save_local("index_storage")
faiss_store.save_local("index_storage_2")

In [23]:
from langchain_community.vectorstores import FAISS
faiss_load=FAISS.load_local("index_storage_2",embeddings,allow_dangerous_deserialization=True)

query="British rule lasted till when"
res=faiss_load.similarity_search(query,k=2)

for i , doc in enumerate(res):
     print(f"\n--- Result {i} ---\n{doc.page_content}")


--- Result 0 ---
British Protectorate (1882–1952)[edit]
Main articles: History of Egypt under the British and History of modern Egypt
Nationalists demonstrating in Cairo, 1919
British indirect rule lasted from 1882, when the British succeeded in defeating the Egyptian Army at Tel el-Kebir in September and took control of the country, to the 1952 Egyptian revolution which made Egypt a republic and when British advisers were expelled.
Muhammad Ali was succeeded briefly by his son Ibrahim (in September 1848), then by a grandson Abbas I (in November 1848), then by Said (in 1854), and Isma'il (in 1863).

--- Result 1 ---
12Now †Namibia
13League of Nations mandate. British Cameroons is now part of †Cameroon and Nigeria, while Tanganyika is part of †Tanzania.
14Self-governing Southern Rhodesia unilaterally declared independence in 1965 (as Rhodesia) and continued as an unrecognised state until the 1979 Lancaster House Agreement. After recognised independence in 1980, Zimbabwe was a member of

In [10]:
##CHROMA STORE 
from langchain_community.vectorstores import Chroma
chroma_store=Chroma.from_documents(documents=final_split,embedding=embeddings,persist_directory="chroma_storage")
chroma_store.persist()

C:\Users\athar\AppData\Local\Temp\ipykernel_41784\3905476134.py:3: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  chroma_store.persist()


In [ ]:
query="What are reseptors in the skin are called"
res=chroma_store.similarity_search_with_score(query)
for i, (doc, score) in enumerate(res):
    print(f"\n--- Result {i+1} (Score: {score}) ---\n{doc.page_content}")


NameError: name 'chroma_store' is not defined